In [1]:
def tokenize(chars):
    "Convert a string of characters into a list of tokens."
    return chars.replace('(', ' ( ').replace(')', ' ) ').split()

program = "(begin (define r 10) (* pi (* r r)))"
print(tokenize(program))

['(', 'begin', '(', 'define', 'r', '10', ')', '(', '*', 'pi', '(', '*', 'r', 'r', ')', ')', ')']


In [2]:
Symbol = str          # A Scheme Symbol is implemented as a Python str
List   = list         # A Scheme List is implemented as a Python list
Number = (int, float) # A Scheme Number is implemented as a Python int or float

In [3]:
def parse(program):
    "Read a Scheme expression from a string."
    return read_from_tokens(tokenize(program))

def read_from_tokens(tokens):
    "Read an expression from a sequence of tokens."
    if len(tokens) == 0:
        raise SyntaxError('unexpected EOF while reading')
    token = tokens.pop(0)
    if '(' == token:
        L = []
        while tokens[0] != ')':
            L.append(read_from_tokens(tokens))
        tokens.pop(0) # pop off ')'
        return L
    elif ')' == token:
        raise SyntaxError('unexpected )')
    else:
        return atom(token)

def atom(token):
    "Numbers become numbers; every other token is a symbol."
    try: return int(token)
    except ValueError:
        try: return float(token)
        except ValueError:
            return Symbol(token)

In [4]:
import math
import operator as op

def standard_env():
    "An environment with some Scheme standard procedures."
    env = Env()
    env.update(vars(math)) # sin, cos, sqrt, pi, ...
    env.update({
        '+':op.add, '-':op.sub, '*':op.mul, '/':op.truediv, 
        '>':op.gt, '<':op.lt, '>=':op.ge, '<=':op.le, '=':op.eq, 
        'abs':     abs,
        'append':  op.add,  
        #'apply':   apply,
        'begin':   lambda *x: x[-1],
        'car':     lambda x: x[0],
        'cdr':     lambda x: x[1:], 
        'cons':    lambda x,y: [x] + y,
        'eq?':     op.is_, 
        'equal?':  op.eq, 
        'length':  len, 
        'list':    lambda *x: list(x), 
        'list?':   lambda x: isinstance(x,list), 
        'map':     map,
        'max':     max,
        'min':     min,
        'not':     op.not_,
        'null?':   lambda x: x == [], 
        'number?': lambda x: isinstance(x, Number),   
        'procedure?': callable,
        'round':   round,
        'symbol?': lambda x: isinstance(x, Symbol),
        #'null': []
    })
    return env

In [44]:
class Procedure(object):
    "A user-defined Scheme procedure."
    def __init__(self, params, body, env):
        self.params, self.body, self.env = params, body, env
    def __call__(self, *args):
        return evaluate(self.body, Env(self.params, args, self.env)) #initialise a temporary environent

class Env(dict):
    "An environment: a dict of {'var':val} pairs, with an outer Env."
    def __init__(self, params=(), args=(), outer=None):
        if outer != None:
            self.update(outer) #the golobal params
        self.update(zip(params, args)) #overwrite with locally defined ones
        self.outer = outer
    def find(self, var):
        "Find the innermost Env where var appears."
        return self if (var in self) else self.outer.find(var)
    
global_env = standard_env()

In [45]:
def evaluate(prog, env=global_env):
    "Evaluate an expression in an environment."
    
    if isinstance(prog, Symbol):      # variable reference
        return env[prog]
    
    elif not isinstance(prog, List):  # constant literal
        return prog        
    
    ### Conditional
    #(if (> 10 3) (- 1 2) (+ 3 4)) -> -1
    elif prog[0] == 'if':
        (_, test, conseq, alt) = prog
        exp = (conseq if evaluate(test, env) else alt)
        return evaluate(exp, env)
    
    ### Definition
    #(define r 3)
    elif prog[0] == 'define':
        (_, var, exp) = prog
        env[var] = evaluate(exp, env)
        
    ### Quotation
    #(quote (+ 1 2)) -> (+ 1 2)
    elif prog[0] == 'quote':          
        (_, exp) = prog
        return exp

    ### Assignment
    #(set! r2 (* r r))
    elif prog[0] == 'set!':
        (_, var, exp) = prog
        env.find(var)[var] = evaluate(exp, env)
        
    ### Lambda expression
    #(lambda (r) (* pi (* r r)))
    elif prog[0] == 'lambda':
        (_,params,body) = prog
        return Procedure(params,body,env)
        
    ### Procedure call
    else:
        proc = evaluate(prog[0], env)
        args = [evaluate(arg, env) for arg in prog[1:]]
        return proc(*args)

In [48]:
evaluate(parse("(define r 10)"))
print(evaluate(parse("(* pi (* r (* r r)))")))
evaluate(parse("(define circle-area (lambda (r) (* pi (* r r))))"))
print(evaluate(parse("(circle-area 20)")))

3141.592653589793
1256.6370614359173


In [47]:
def repl(prompt='lis.py> '):
    "A prompt-read-eval-print loop."
    while True:
        cmd = input(prompt)
        if cmd != '':
            val = evaluate(parse(cmd))
            if val is not None: 
                print(schemestr(val))

def schemestr(exp):
    "Convert a Python object back into a Scheme-readable string."
    if  isinstance(exp, list):
        return '(' + ' '.join(map(schemestr, exp)) + ')' 
    else:
        return str(exp)

In [ ]:
repl()

# Questions

* How does env get updated in the recursive calls within evaluate?
* What is a macro?

# Lessons learnt

* Lambda calculi have their own (local) environments (wasnt clear until you need to code it).
* At its core, it's a translator.

Code stolen from http://norvig.com/lispy.html.

### TODO

* Make some sort of finite state machine/diagram
* http://norvig.com/lispy2.html